Pra instalar o Jupyter é so seguir a descricao do video (la ele ensina o basicao dessa IDE): https://www.youtube.com/watch?v=_eK0z5QbpKA&t=926s

In [41]:
#Packages to be installed (run only once)

#This package is used to automate tasks in chrome
!pip install selenium
#This package is used to automated keyboard and mouse functions
!pip install pyautogui
#This package is used to copy and paste special characters
!pip install pyperclip

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.53-py3-none-any.whl size=36613 sha256=c668383058469b1a6b8f8b25e374d6012e9626369b599ce2fb646a9335ce15d4
  Stored in directory: c:\users\bruno silva de lima\appdata\local\pip\cache\wheels\d8\97\e4\d2edca92a87d3b5fbfb527264750a17b4ba297b9a7cab6e67f
  Created wheel for pygetwindow: filename=PyGetWindow-0.0.9-py3-none-any.whl

Before running the code it is important to download the chrome webdriver: https://chromedriver.chromium.org/downloads
Open the location in which Python is installed, for that open Anaconda prompt. In my case "C:\ProgramData\Anaconda3"

In [56]:
#Import Selenium package (https://selenium-python.readthedocs.io/)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Import pyautogui to control the keyboard and the mouse
import pyautogui

#Import pyperclip to copy and paste special characters
import pyperclip

#Import time to control timing
import time

#Make the code waits x seconds between each comand line
pyautogui.PAUSE = 0.5

#Open a new window in Chrome
new_tab = webdriver.Chrome()
#Open an specific website
new_tab.get("https://www.google.com.br/")

#Access an specific location of the site and type a message
#To find the location of the website you want, right click on it, click on "investigate", type "ctrl + shift + c", click on the part you want (it will be highlighted in blue on the right side), right click on the highlighted blue region -> copy -> copy xpath
new_tab.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('1 Euro in Reais')
#Access the same location and press enter
new_tab.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#Getting the wanted result
#To find the attribute you want just inspect the page like before but instead of copying the xpath read the highlighted blue region and check the data you want
euro_in_reais_str = new_tab.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
euro_in_reais= float(euro_in_reais_str)

#Open email to send information (in this case hotmail)
new_tab.get("https://outlook.live.com/mail/0/")

#Waits you to sign in in the email
print('After the log in press enter to continue')
x = input()

#Selecting the write email button
#to copy and past the pyperclip package will be used to avoid errors with special characters
xpath = '//*[@id="id__6"]'
pyperclip.copy(xpath)
new_tab.find_element_by_xpath(pyperclip.paste()).click()

#The next commands uses pyautogui to control the keyboard to send the email
#After the login you need to make sure that alt + tab goes to the email window
pyautogui.hotkey('alt','tab')

#use time to wait untill the proccess ends
time.sleep(3)
#typing the name of the recipient and choosing it (the waiting time is the proccess to save the recipient)
pyautogui.write('*************type the email recipient here ****************')
pyautogui.press('tab')
time.sleep(2)

#Change to the subject scope and wrting the subject
pyautogui.press('tab')
pyautogui.write('Value of euro in reais.')

#Changing to text scope and typing the value of the euro
pyautogui.press('tab')
pyautogui.write('Each euro is worth {0:.2f} reais today.'.format(euro_in_reais))

#Finding the right place to send the email
pyautogui.press('tab')
pyautogui.press('tab')
pyautogui.press('tab')
pyautogui.press('enter')

#close the window
new_tab.quit()

C:\Users\BRUNOS~1\AppData\Local\Temp/ipykernel_13328/1125289809.py:24: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  new_tab.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('1 Euro in Reais')
C:\Users\BRUNOS~1\AppData\Local\Temp/ipykernel_13328/1125289809.py:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  new_tab.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
C:\Users\BRUNOS~1\AppData\Local\Temp/ipykernel_13328/1125289809.py:30: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  euro_in_reais_str = new_tab.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


After the log in press enter to continue



C:\Users\BRUNOS~1\AppData\Local\Temp/ipykernel_13328/1125289809.py:44: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  new_tab.find_element_by_xpath(pyperclip.paste()).click()
